# The ELLA REST API

Ella has a large REST API that can be used to fetch all kinds of data.

In [1]:
import requests
import json
from pprint import pprint

## Querying the ELLA API

In order to get the REST API you'll need to know what port Ella is running on. 

This can get a bit tricky because you need to pay attention to whether or not you are running within the docker container or the port exposed outside.

In this case I am running in the jupyterhub notebook provided within the `ella-admin` jupyterhub stack. This means that I am using the network created by docker, so I query ella using the service name `ella-web` on port `5000`.

(For the sake of brevity I got rid of some of the variables in the ella-web service.)

```yaml
  ella-web:
    image: dabbleofdevops/ella:1.11.1
    ports:
      # External:Internal
      # Expose the port running in the container as 5000 to my localhost as 5001
      - 5001:5000
```

### Query Internally from within a Docker Container

You could exec (`docker-compose exec -f local.yml ella-web` or `docker-compose exec -f local.yml jupyter`) into a docker container, or use the notebook to query the Ella API. If you do that you should use the url `http://{service-name}/{internal-port}`.

### Query Externally from the Docker Host

If you want to run this command externally, such as if you are using Postman, you need to use the external url. `http://localhost:5001` or `http://{ip-of-server}/{external-port}`.

In [2]:
response = requests.get('http://ella-web:5000/api/v1/specs')
content = response.content.decode('utf-8')
api_spec_data = json.loads(content)

print('--------------------------')
pprint(api_spec_data.keys())
print('--------------------------')
pprint(api_spec_data['definitions'].keys())
print('--------------------------')
pprint(list(api_spec_data['paths'].keys()))

--------------------------
dict_keys(['info', 'paths', 'tags', 'swagger', 'definitions', 'parameters'])
--------------------------
dict_keys(['Analysis', 'AnalysisInterpretation', 'AlleleInterpretation', 'Interpretation', 'Allele', 'Reference', 'ReferenceAssessment', 'AlleleAssessment', 'AlleleAssessmentInput', 'AlleleReport', 'User', 'Classification', 'Rule', 'Genepanel', 'Annotation', 'CustomAnnotation', 'Genotype'])
--------------------------
['/api/v1/acmg/alleles/',
 '/api/v1/acmg/classifications/',
 '/api/v1/alleles/',
 '/api/v1/alleles/by-gene/',
 '/api/v1/alleles/{allele_id}/analyses/',
 '/api/v1/alleleassessments/{aa_id}/',
 '/api/v1/alleleassessments/',
 '/api/v1/allelereports/',
 '/api/v1/allelereports/{ar_id}/',
 '/api/v1/analyses/',
 '/api/v1/analyses/{analysis_id}/',
 '/api/v1/broadcasts/',
 '/api/v1/customannotations/',
 '/api/v1/config/',
 '/api/v1/filterconfigs/{filterconfig_id}',
 '/api/v1/geneassessments/',
 '/api/v1/genepanels/',
 '/api/v1/genepanels/{name}/{version

## Authenticating Against the REST API

In order to authenticate against the REST API we'll have to pass in our username and password. If you ran the dev stack as 

```bash
make dev
make load
```

You should already have a test user setup.

(Please note that the dev stack is for dev purposes only and you should change your own usernames and passwords!)

Once we've authenticated we should have a cookie called 'AuthenticationToken'. Using the Python Requests session this is saved for us. Otherwise you will have to add the token to your cookies manually.

In [3]:
session = requests.Session()

url = 'http://ella-web:5000'
user_data = {"username": "testuser1", "password": "Password#123"}
session.post('{}/api/v1/users/actions/login/'.format(url), json=user_data)
print(session.cookies.get_dict())

{'AuthenticationToken': '0b5a3cc5679cd7bcdb82206b46b4d21c6546b1ba835568e44047fc5910385a7c'}


Now that we're authenticated we can start running queries! 

## Run Authenticated Queries

In [4]:
response = session.get('{}/api/v1/workflows/analyses/2/interpretations/2/'.format(url))
print(session.cookies.get_dict())

{'AuthenticationToken': '0b5a3cc5679cd7bcdb82206b46b4d21c6546b1ba835568e44047fc5910385a7c'}


In [5]:
pprint(json.loads(response.content.decode('utf-8')))

{'date_last_update': '2020-11-28T06:44:25.638124+00:00',
 'finalized': True,
 'genepanel_name': 'HBOCUTV',
 'genepanel_version': 'v01',
 'id': 2,
 'state': {'allele': {'3': {'allele_id': 3,
                            'alleleassessment': {'allele_id': 3,
                                                 'evaluation': {'acmg': {'suggested': [{'code': 'REQ_GP_last_exon_not_important',
                                                                                        'match': ['LENI'],
                                                                                        'op': '$in',
                                                                                        'source': 'genepanel.last_exon_important'},
                                                                                       {'code': 'REQ_GP_LOF_missense',
                                                                                        'match': ['ANY'],
                                                  

                            'allelereport': {'copiedFromId': 5,
                                             'evaluation': {'comment': ''}},
                            'analysis': {'comment': '',
                                         'notrelevant': None,
                                         'verification': 'verified'},
                            'referenceassessments': [],
                            'report': {'included': True},
                            'workflow': {'reviewed': True}},
                      '8': {'allele_id': 8,
                            'alleleassessment': {'allele_id': 8,
                                                 'evaluation': {'acmg': {'suggested': [{'code': 'REQ_GP_last_exon_not_important',
                                                                                        'match': ['LENI'],
                                                                                        'op': '$in',
                                                 

In [6]:
response = session.get('{}/api/v1/analyses/2'.format(url))
pprint(json.loads(response.content.decode('utf-8')))

{'date_deposited': '2020-10-19T10:46:43.419040+00:00',
 'date_requested': None,
 'genepanel': {'name': 'HBOCUTV', 'version': 'v01'},
 'id': 2,
 'interpretations': [{'analysis_id': 2,
                      'date_last_update': '2020-11-28T06:44:25.638124+00:00',
                      'finalized': True,
                      'genepanel_name': 'HBOCUTV',
                      'genepanel_version': 'v01',
                      'id': 2,
                      'status': 'Done',
                      'user': {'abbrev_name': 'H. Ibsen',
                               'active': True,
                               'first_name': 'Henrik',
                               'full_name': 'Henrik Ibsen',
                               'id': 1,
                               'last_name': 'Ibsen',
                               'user_group_name': 'testgroup01',
                               'username': 'testuser1'},
                      'user_id': 1,
                      'workflow_status': 'Interpretatio

## Ella REST API Analysis Overview

This is the sequence of events the REST API goes through when generating the Allele Interpretation Interface.

You can get this sequence by checking out the ella-web logs when you bring up an analysis page.

```
#GET /api/v1/workflows/analyses/2/interpretations/2/alleles/ 
GET /workflows/analyses/2/ HTTP/1.1"

GET /api/v1/config/
GET /api/v1/users/currentuser/
GET /api/v1/workflows/analyses/2/stats/
GET /api/v1/analyses/2/
GET /api/v1/workflows/analyses/2/filterconfigs -
GET /api/v1/workflows/analyses/2/filterconfigs/ -
GET /api/v1/workflows/analyses/2/interpretations/  -
GET /api/v1/workflows/analyses/2/interpretations/2/filteredalleles/?filterconfig_id=1 -
GET /api/v1/workflows/analyses/2/genepanels/HBOCUTV/v01/?allele_ids=3%2C4%2C5%2C6%2C7%2C8  -
GET /api/v1/workflows/analyses/2/interpretations/2/alleles/?allele_ids=3%2C4%2C5%2C6%2C7%2C8&current=true&filterconfig_id=1 HTTP/1.1" 200 -
GET /api/v1/workflows/analyses/2/collisions/?allele_ids=3%2C4%2C5%2C6%2C7%2C8 HTTP/1.1" 200 -
GET /api/v1/references/?q=%7B%22id%22%3A%5B%5D%7D HTTP/1.1" 200 -
GET /api/v1/attachments/?q=%7B%22id%22%3A%5B%5D%7D HTTP/1.1" 200 -
GET /api/v1/references/?q=%7B%22pubmed_id%22%3A%5B17453335%2C8589730%2C18607349%2C28324225%2C18092194%2C9245992%2C10070953%2C22430266%2C20927582%2C25525159%2C21356067%2C16760289%2C16847550%2C18844490%2C28492532%2C19471317%2C10923033%5D%7D HTTP/1.1" 200 -
POST - http://localhost:5001/api/v1/acmg/alleles/ - {"allele_ids":["3","4","5","6","7","8"],"gp_name":"HBOCUTV","gp_version":"v01","referenceassessments":[]} - 4705 - 650ms
"POST /api/v1/acmg/alleles/ HTTP/1.1" 200 -
```


In [7]:
#query_url = 'api/v1/workflows/analyses/2'
#response = session.get('{}/{}'.format(url, query_url))
#pprint(json.loads(response.content.decode('utf-8')))

In [8]:
query_url = 'api/v1/config/'
response = session.get('{}/{}'.format(url, query_url))
#pprint(json.loads(response.content.decode('utf-8')))

In [9]:
query_url = 'api/v1/workflows/analyses/2/stats/'
response = session.get('{}/{}'.format(url, query_url))
pprint(json.loads(response.content.decode('utf-8')))

{'allele_count': 6}


In [10]:
query_url = 'api/v1/analyses/2/'
response = session.get('{}/{}'.format(url, query_url))
pprint(json.loads(response.content.decode('utf-8')))

{'date_deposited': '2020-10-19T10:46:43.419040+00:00',
 'date_requested': None,
 'genepanel': {'name': 'HBOCUTV', 'version': 'v01'},
 'id': 2,
 'interpretations': [{'analysis_id': 2,
                      'date_last_update': '2020-11-28T06:44:25.638124+00:00',
                      'finalized': True,
                      'genepanel_name': 'HBOCUTV',
                      'genepanel_version': 'v01',
                      'id': 2,
                      'status': 'Done',
                      'user': {'abbrev_name': 'H. Ibsen',
                               'active': True,
                               'first_name': 'Henrik',
                               'full_name': 'Henrik Ibsen',
                               'id': 1,
                               'last_name': 'Ibsen',
                               'user_group_name': 'testgroup01',
                               'username': 'testuser1'},
                      'user_id': 1,
                      'workflow_status': 'Interpretatio

In [11]:
query_url = 'api/v1/workflows/analyses/2/filterconfigs/'
response = session.get('{}/{}'.format(url, query_url))
#pprint(json.loads(response.content.decode('utf-8')))

In [12]:
## Get some filtered alleles

query_url = 'api/v1/workflows/analyses/2/interpretations/2/filteredalleles/'
payload = {'filterconfig': 1}
response = session.get('{}/{}'.format(url, query_url), params=payload)
response = json.loads(response.content.decode('utf-8'))
pprint(response)
allele_ids = response['allele_ids']
allele_ids

{'allele_ids': [3, 4, 5, 6, 7, 8],
 'excluded_allele_ids': {'classification': [],
                         'consequence': [],
                         'frequency': [],
                         'gene': [],
                         'inheritancemodel': [],
                         'ppy': [],
                         'quality': [],
                         'region': [],
                         'segregation': []}}


[3, 4, 5, 6, 7, 8]

In [13]:
# /api/v1/workflows/analyses/2/genepanels/HBOCUTV/v01/?allele_ids=3%2C4%2C5%2C6%2C7%2C8 

# TODO Get the Genepanel from the analysis def
query_url = 'api/v1/workflows/analyses/2/genepanels/HBOCUTV/v01/'
payload = {'allele_ids': allele_ids}
response = session.get('{}/{}'.format(url, query_url), params=payload)
response = json.loads(response.content.decode('utf-8'))
pprint(response)

{'geneassessments': [],
 'name': 'HBOCUTV',
 'phenotypes': [{'description': '{Breast cancer, male, susceptibility to}',
                 'gene': {'ensembl_gene_id': 'ENSG00000139618',
                          'hgnc_id': 1101,
                          'hgnc_symbol': 'BRCA2',
                          'omim_entry_id': 600185},
                 'id': 6,
                 'inheritance': 'AD',
                 'omim_id': 114480},
                {'description': '{Breast-ovarian cancer, familial, 2}',
                 'gene': {'ensembl_gene_id': 'ENSG00000139618',
                          'hgnc_id': 1101,
                          'hgnc_symbol': 'BRCA2',
                          'omim_entry_id': 600185},
                 'id': 2,
                 'inheritance': 'AD',
                 'omim_id': 612555},
                {'description': 'Fanconi anemia, complementation group D1',
                 'gene': {'ensembl_gene_id': 'ENSG00000139618',
                          'hgnc_id': 1101,
     

In [14]:
# /api/v1/workflows/analyses/2/interpretations/2/alleles/
# ?allele_ids=3%2C4%2C5%2C6%2C7%2C8&current=true&filterconfig_id=1

query_url = 'api/v1/workflows/analyses/2/interpretations/2/alleles/'
# Allele Ids needs to be a string with comma separated values, not a list!
payload = {'allele_ids': ','.join([str(x) for x in allele_ids]), 
           'current': True, 
           'filterconfig_id': 1
          }
response = session.get('{}/{}'.format(url, query_url), params=payload)

print(response.url)
response = json.loads(response.content.decode('utf-8'))
pprint(len(response))
pprint(response)


http://ella-web:5000/api/v1/workflows/analyses/2/interpretations/2/alleles/?allele_ids=3%2C4%2C5%2C6%2C7%2C8&current=True&filterconfig_id=1
6
[{'allele_assessment': {'allele_id': 3,
                        'analysis_id': 2,
                        'annotation_id': 3,
                        'attachment_ids': [],
                        'classification': '4',
                        'custom_annotation_id': None,
                        'date_created': '2020-10-19T10:49:44.630277+00:00',
                        'date_superceeded': None,
                        'evaluation': {'acmg': {'included': [{'code': 'PVS1',
                                                              'comment': '',
                                                              'match': None,
                                                              'op': None,
                                                              'source': 'suggested',
                                                              'uuid'

                                                    {'clinical_significance_descr': 'Pathogenic',
                                                     'last_evaluated': '02/10/2015',
                                                     'rcv': 'SCV000327129',
                                                     'submitter': 'CIMBA',
                                                     'traitnames': '',
                                                     'variant_id': ''},
                                                    {'clinical_significance_descr': 'Pathogenic',
                                                     'last_evaluated': '23/09/2016',
                                                     'rcv': 'SCV000186926',
                                                     'submitter': 'Ambry '
                                                                  'Genetics',
                                                     'traitnames': 'Hereditary '
                              

                                 'user_group_name': 'testgroup01',
                                 'username': 'testuser1'},
                        'user_id': 1,
                        'usergroup': {'id': 1, 'name': 'testgroup01'},
                        'usergroup_id': 1},
  'allele_report': {'allele_id': 5,
                    'analysis_id': 2,
                    'date_created': '2020-10-19T10:50:14.366927+00:00',
                    'date_superceeded': None,
                    'evaluation': {'comment': ''},
                    'id': 3,
                    'previous_report_id': None,
                    'seconds_since_update': 3448176.81548,
                    'user': {'abbrev_name': 'H. Ibsen',
                             'active': True,
                             'first_name': 'Henrik',
                             'full_name': 'Henrik Ibsen',
                             'id': 1,
                             'last_name': 'Ibsen',
                             'user_group_

                                                  'OTH': 894,
                                                  'SAS': 16492}},
                                 'GNOMAD_EXOMES': {'count': {'AFR': 0,
                                                             'AMR': 0,
                                                             'ASJ': 0,
                                                             'EAS': 0,
                                                             'FIN': 5,
                                                             'Female': 4,
                                                             'G': 5,
                                                             'Male': 1,
                                                             'NFE': 0,
                                                             'OTH': 0,
                                                             'SAS': 0},
                                                   'filter': {'G': ['PASS']},
          

                                                     'last_evaluated': '18/10/2016',
                                                     'rcv': 'RCV000256555',
                                                     'submitter': '',
                                                     'traitnames': 'Breast-ovarian '
                                                                   'cancer, '
                                                                   'familial 2',
                                                     'variant_id': '52900'},
                                                    {'clinical_significance_descr': 'Pathogenic',
                                                     'last_evaluated': '02/10/2015',
                                                     'rcv': 'SCV000328170',
                                                     'submitter': 'CIMBA',
                                                     'traitnames': '',
                                           

                                                            'NFE': 0,
                                                            'OTH': 0},
                                                    'num': {'AFR': 8736,
                                                            'AMR': 838,
                                                            'ASJ': 302,
                                                            'EAS': 1622,
                                                            'FIN': 3494,
                                                            'Female': 13858,
                                                            'G': 30980,
                                                            'Male': 17122,
                                                            'NFE': 15008,
                                                            'OTH': 980}},
                                 'esp6500': {'freq': {'AA': 0.0, 'EA': 0.0003}},
                                 'inDB': {'

In [15]:
len(response)

6

In [16]:
# api/v1/workflows/analyses/2/collisions/?allele_ids=3%2C4%2C5%2C6%2C7%2C8
query_url = 'api/v1/workflows/analyses/2/collisions/'
payload = {'allele_ids': allele_ids}
response = session.get('{}/{}'.format(url, query_url), params=payload)
response = json.loads(response.content.decode('utf-8'))
pprint(response)

[]


In [17]:
#POST - http://localhost:5001/api/v1/acmg/alleles/ - 
json_data = {"allele_ids":["3","4","5","6","7","8"],"gp_name":"HBOCUTV","gp_version":"v01","referenceassessments":[]}

query_url = 'api/v1/acmg/alleles/' 
response = session.post('{}/{}'.format(url, query_url), json=json_data)
response = json.loads(response.content.decode('utf-8'))
pprint(response)

{'3': {'codes': [{'code': 'REQ_GP_last_exon_not_important',
                  'match': ['LENI'],
                  'op': '$in',
                  'source': 'genepanel.last_exon_important',
                  'value': ['LENI']},
                 {'code': 'REQ_GP_LOF_missense',
                  'match': ['ANY'],
                  'op': '$in',
                  'source': 'genepanel.disease_mode',
                  'value': ['ANY']},
                 {'code': 'REQ_not_in_last_exon',
                  'match': ['no'],
                  'op': '$in',
                  'source': 'transcript.in_last_exon',
                  'value': ['no']},
                 {'code': 'REQ_null_variant',
                  'match': ['stop_gained'],
                  'op': '$in',
                  'source': 'transcript.consequences',
                  'value': ['start_lost',
                            'initiator_codon_variant',
                            'transcript_ablation',
                            'splice